In [12]:
import numpy as np
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import lightgbm as lgb
from sklearn import svm
import xgboost as xgb
import catboost
from sklearn.neural_network import MLPRegressor
import optuna

from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [13]:
def load_data(y):
    path = '../../data/std_data/'
    x_train_std = pd.read_pickle(path +'train/{}_x.pkl'.format(str(y))).values
    x_test_std = pd.read_pickle(path +'test/{}_x.pkl'.format(str(y))).values
    y_train = pd.read_pickle(path +'train/{}_y.pkl'.format(str(y))).values
    y_test = pd.read_pickle(path +'test/{}_y.pkl'.format(str(y))).values
    features = pd.read_pickle(path +'train/{}_x.pkl'.format(str(y))).columns
    return x_train_std, x_test_std, y_train, y_test, features

In [14]:
def plot_roc_curve(fpr, tpr, auc):
    # ROC曲線をプロット
    plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
    plt.legend()
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)
    plt.show()

In [15]:
def train():
    
    # 予測した確率全体を格納
    probs_all_lr = np.array([])
    probs_all_lda = np.array([])
    probs_all_sv = np.array([])
    probs_all_lgbm = np.array([])
    probs_all_xgb = np.array([])
    probs_all_cb = np.array([])
    probs_all_mlp = np.array([])
    
    y_true_all = np.array([])
    
    for y in range(1978, 2020):
        
        # データの生成
        x_train_std, x_test_std, y_train, y_test, features = load_data(y)
        y_true_all = np.hstack((y_true_all, y_test))
       
        # logistic regression
        lr = LogisticRegression(class_weight="balanced", solver="liblinear",  penalty="l2", C=0.0001) # ロジスティック回帰モデルのインスタンスを作成
        lr.fit(x_train_std, y_train) # ロジスティック回帰モデルの重みを学習
        probs_lr = lr.predict_proba(x_test_std)[:,1]
        probs_all_lr = np.hstack((probs_all_lr, probs_lr))
        
        # LDA
        lda = LDA(solver="eigen", shrinkage=1).fit(x_train_std,  y_train)
        probs_lda = lda.predict_proba(x_test_std)[:,1]
        probs_all_lda = np.hstack((probs_all_lda, probs_lda))
        
        # svm
        sv = svm.SVR(kernel="sigmoid",
                                     degree=4,
                                     gamma=0.043502212815589775,
                                     coef0=0.20190829020616494,
                                     tol=0.0001,
                                     C=0.000245786293391316,
                                     epsilon=0.3056167642389302,
                                    verbose=False,)
        sv.fit(x_train_std, y_train)
        probs_sv = sv.predict(x_test_std)
        probs_all_sv = np.hstack((probs_all_sv, probs_sv))
        
        # xgb
        xgboost = xgb.XGBRegressor(silent= True, 
                               max_depth=4,
                               learning_rate=0.12765177534095626,
                               n_estimators = 46,
                               gamma=0.060805284848630535,
                               reg_lambda=4.995675788308118,
                               reg_alpha=2.1912254426545754,
                               sub_sample=0.45297631180790854,
                               scale_pos_weight=1.1672978934986058)
        xgboost.fit(x_train_std, y_train)
        probs_xgb = xgboost.predict(x_test_std)
        probs_all_xgb = np.hstack((probs_all_xgb, probs_xgb))
        
        
        
        # lgbm
        lgbm = lgb.LGBMRegressor(
            random_state=0,
            verbosity=-1,
            bagging_seed=0,
            boost_from_average='true',
            metric='auc',
            bagging_freq=4,
            min_data_in_leaf=21,
            max_depth=13,
            learning_rate=0.08731913651405197,
            n_estimators=3394,
            subsample=0.7054763057027115,
            num_leaves=438,
            reg_lambda=0.9377125325944119,  
        )
        
        lgbm.fit(x_train_std, y_train)
        probs_lgbm = lgbm.predict(x_test_std)
        probs_all_lgbm = np.hstack((probs_all_lgbm, probs_lgbm))
        
        # catboost
        cb = catboost.CatBoostRegressor(
            iterations=258,
            depth=2,
            learning_rate=0.019083573879517587,
            random_strength=84,
            bagging_temperature=0.3233702745357832,
            od_type="Iter",
            od_wait=32, 
            logging_level='Silent')
        cb.fit(x_train_std, y_train)
        probs_cb = cb.predict(x_test_std)
        probs_all_cb = np.hstack((probs_all_cb, probs_cb))   
        
        # mlp
        mlp = MLPRegressor(hidden_layer_sizes=(32,),
                           activation='relu',
                           solver='adam',
                           alpha=4.76324733221396,
                           batch_size='auto',
                           learning_rate='constant', 
                           learning_rate_init=0.0012043271455668674, 
                           power_t=0.5,
                           max_iter=1000, 
                           shuffle=True,
                           random_state=0, 
                           tol=0.0001, 
                           verbose=False, 
                           warm_start=False, 
                           momentum=0.9,
                           nesterovs_momentum=True, 
                           early_stopping=False, 
                           validation_fraction=0.1, 
                           beta_1=0.022158342014810775, 
                           beta_2= 0.7802116425099002,
                           epsilon=1e-08,
                           )
        mlp.fit(x_train_std, y_train)
        probs_mlp = mlp.predict(x_test_std)
        probs_all_mlp = np.hstack((probs_all_mlp, probs_mlp))   
    
    auc_lr = roc_auc_score(y_true_all, probs_all_lr)
    auc_lda = roc_auc_score(y_true_all, probs_all_lda)
    auc_sv = roc_auc_score(y_true_all, probs_all_sv)
    auc_xgb = roc_auc_score(y_true_all, probs_all_xgb)
    auc_lgbm = roc_auc_score(y_true_all, probs_all_lgbm)
    auc_cb = roc_auc_score(y_true_all, probs_all_cb)
    auc_mlp = roc_auc_score(y_true_all, probs_all_mlp)
    
    all_models_probs = [probs_all_lr, probs_all_lda, probs_all_sv, probs_all_xgb, probs_all_lgbm, probs_all_cb, probs_all_mlp]
    
    print("len: {0} ".format(len(y_true_all) ))

    print("AUC LR: ",auc_lr)
    print("AUC LDA: ",auc_lda)
    print("AUC svm: ",auc_sv)
    print("AUC xgb: ",auc_xgb)
    print("AUC lgbm: ", auc_lgbm)
    print("AUC CB: ",auc_cb)
    print("AUC MLP: ",auc_mlp)
    print()
    return y_true_all, all_models_probs

In [16]:
def optimize(trial, y_true_all, all_models_probs):
    
    w_lr = trial.suggest_uniform('w_lr', 0.0, 1.0)
    w_lda = trial.suggest_uniform('w_lda', 0.0, 1.0)
    w_sv = trial.suggest_uniform('w_sv', 0.0, 1.0)
    w_xgb = trial.suggest_uniform('w_xgb', 0.0, 1.0)
    w_lgbm = trial.suggest_uniform('w_lgbm', 0.0, 1.0)
    w_cb = trial.suggest_uniform('w_cb', 0.0, 1.0)
    w_mlp = trial.suggest_uniform('w_mlp', 0.0, 1.0)
    
    # 各モデルの重み
    w = [w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb, w_mlp]
    
    print("[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] = ", w)
    
    # 各モデル結果の重み付き平均
    probs_weighted_average = np.array([0 for i in range(len(y_true_all))], dtype='float64')
    for probs, weight in zip(all_models_probs, w):
        probs_weighted_average += (probs * weight)

    auc = roc_auc_score(y_true_all, probs_weighted_average)
    
    fpr, tpr, thresholds = roc_curve(y_true_all, probs_weighted_average)
    
#     plot_roc_curve(fpr, tpr, auc)

    print("AUC all: ",auc)
    print()
    return -auc

In [17]:
def main():
    np.set_printoptions(precision=3, suppress=True)
    
    y_true_all, all_models_probs = train()
    
    study = optuna.create_study()
    study.optimize(lambda trial: optimize(trial, y_true_all, all_models_probs), n_trials=10000)
    print(study.best_trial)

In [ ]:
main()

len: 211 
AUC LR:  0.7482389405466329
AUC LDA:  0.7418991265145111
AUC svm:  0.794590025359256
AUC xgb:  0.7607072414764722
AUC lgbm:  0.7602141448295294
AUC CB:  0.7561284868977177
AUC MLP:  0.777120315581854

[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.31307006847012775, 0.5889169786171274, 0.16460101975508412, 0.12029298585009207, 0.4399541623760681, 0.3684840358078255, 0.24599068492392173]
AUC all:  0.7762750070442378



[I 2019-08-23 15:02:40,197] Finished trial#0 resulted in value: -0.7762750070442378. Current best value is -0.7762750070442378 with parameters: {'w_lr': 0.31307006847012775, 'w_lda': 0.5889169786171274, 'w_sv': 0.16460101975508412, 'w_xgb': 0.12029298585009207, 'w_lgbm': 0.4399541623760681, 'w_cb': 0.3684840358078255, 'w_mlp': 0.24599068492392173}.


[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.4019556512673871, 0.4890357634931384, 0.8594500228123739, 0.1734489608838644, 0.67325052919427, 0.5691324443902135, 0.07007164060759519]
AUC all:  0.7776838546069315



[I 2019-08-23 15:02:40,304] Finished trial#1 resulted in value: -0.7776838546069315. Current best value is -0.7776838546069315 with parameters: {'w_lr': 0.4019556512673871, 'w_lda': 0.4890357634931384, 'w_sv': 0.8594500228123739, 'w_xgb': 0.1734489608838644, 'w_lgbm': 0.67325052919427, 'w_cb': 0.5691324443902135, 'w_mlp': 0.07007164060759519}.


[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.41020196184342417, 0.15359074336982093, 0.43741378587869584, 0.807788035351272, 0.7242958660440676, 0.525129099981392, 0.6433682796209738]
AUC all:  0.7841645533953226



...

[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.23077771933794833, 0.10474640897126881, 0.8530365376131398, 0.28338271043327307, 0.2941227050552015, 0.05900048394493101, 0.8785983850218375]
AUC all:  0.792617638771485



[I 2019-08-23 15:32:12,261] Finished trial#1262 resulted in value: -0.792617638771485. Current best value is -0.7944491406029868 with parameters: {'w_lr': 0.10871327127959096, 'w_lda': 0.09215055112748587, 'w_sv': 0.7804052045018608, 'w_xgb': 0.2391208356139352, 'w_lgbm': 0.25423905260387053, 'w_cb': 0.002701662944262521, 'w_mlp': 0.9349433622393312}.


[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.12734128791743202, 0.09288579934271994, 0.7945947842082676, 0.20168579753438087, 0.2508828871096862, 0.021051250494325727, 0.5278758836965138]
AUC all:  0.7890955198647506



[I 2019-08-23 15:32:15,039] Finished trial#1263 resulted in value: -0.7890955198647506. Current best value is -0.7944491406029868 with parameters: {'w_lr': 0.10871327127959096, 'w_lda': 0.09215055112748587, 'w_sv': 0.7804052045018608, 'w_xgb': 0.2391208356139352, 'w_lgbm': 0.25423905260387053, 'w_cb': 0.002701662944262521, 'w_mlp': 0.9349433622393312}.


[w_lr, w_lda, w_sv, w_xgb, w_lgbm,w_cb ] =  [0.09372669610693518, 0.029111783945742237, 0.45003480929033707, 0.2327698567096353, 0.20231556388223332, 0.0009361397916132358, 0.8260138081789649]
AUC all:  0.7923358692589461



In [ ]:
# best value is -0.7938856015779093 with parameters: {'w_lr': 0.051923716713709225, 'w_lda': 0.09981452061167402, 'w_sv': 0.39582318427331886, 'w_xgb': 0.2641194598554109, 'w_lgbm': 0.2813690772599942, 'w_cb': 0.10843852001006549, 'w_mlp': 0.8979452068842029}.